In [5]:
import cv2 as cv
import numpy as  np
import dlib
from ultralytics import YOLO


In [7]:
trackers = []

confThreshold = 0.5
nmsThreshold =  0.5
inpWidth = 416
inpHeight = 416

classFile = "coco.names"
classes = None

with open(classFile, 'rt') as f:
    classes = f.read().rstrip('\n').split('\n')
    
modelConfigurations = "yolov4-tiny.cfg"
modelWeights = "yolov4-tiny.weights"

net = cv.dnn.readNetFromDarknet(modelConfigurations, modelWeights)
net.setPreferableBackend(cv.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv.dnn.DNN_TARGET_CPU)

def findCenter(x,y,w,h):
    cx = int((x+w)/2)
    cy = int((y+h)/2)
    return cx, cy

def pointInRect(x,y,w,h,cx,cy):
    if ((x < cx) &  (cx < x+w)) & ((y < cy) & (cy< y+h)):
        return True 
    return False

def rectT_to_bb(rect):
    x = rect.left()
    y = rect.top()
    w = rect.right() - x
    h = rect.bottom() -y
    return (x,y,w,h)

def getOutputNames(net):
    layersNames = net.getLayerNames()
    return [layersNames[i[0]-1] for i in net.getUnconnectedOutLayers()]


def postProcess(frame,outs):
    global inCount, Font,count,SKIP_FRAMES,outCount
    
    frameHeight = frame.shape[0]
    frameWidth = frame.shape[1]
    
    classIds = []
    confidences = []
    boxes = []
    
    for out in outs:
        for detection in out: 
            scores = detection[5:]
            classId = np.argmax(scores)
            confidence = scores[classId]
            if confidence > confThreshold:
                center_x =  int(detection[0] * frameWidth)
                center_y =  int(detection[1] * frameHeight)
                width =  int(detection[2] * frameWidth)
                height =  int(detection[3] * frameHeight)
                left =  int(center_x - width/2)
                top =  int(center_y - height/2)
                classIds.append(classId)
                confidences.append(float(confidence))
                boxes.append([left, top, width, height])
                
                
    indices = cv.dnn.NMSBoxes(boxes, confidences, confThreshold, nmsThreshold)
    
    trackers_to_del = []
    for tid, trackersid in enumerate(trackers):
        trackingQuality = trackers[0].update(frame_cropped)
        if trackingQuality < 5:
            trackers_to_del.append(tid[0])
            
    try:
        for trackersid in trackers_to_del:
            trackers.pop(trackersid)
    except IndexError:
        pass
    if len(indices) > 0:
        indices.flatten()
        for i in indices:
            i = i[0]
            box = boxes[i]
            left , top, width, height = box
            classId, conf, left,top,right, bottom = classIds[i], confidences[i], left, top, left+width, top+height
            
            rect = dlib.rectangle(left,top,right,bottom)
            (x,y,w,h) = rectT_to_bb(rect)
            
            tracking = False     
            for trackersid in trackers:
                pos = trackersid[0].get_position()
                startX = int(pos.left())
                startY = int(pos.top())
                endX = int(pos.right())
                endY = int(pos.bottom())
                tx, ty = findCenter(startX,startY,endX,endY)
                
                t_location = pointInRect(x,y,w,h,tx,ty)
                if t_location:
                    tracking = True
                
            if not tracking:
                tracker = dlib.correlation_tracker()
                tracker.start_track(frame_cropped,rect)
                trackers.append((tracker, frame_cropped))
                
                
    for num, trackersid in enumerate(trackers):
        pos = trackersid[0].get_position()
        startX = int(pos.left())
        startY = int(pos.top())
        endX = int(pos.right())
        endY = int(pos.bottom())        
        
        cv.rectangle(frame, (startX, startY), (endX, endY), (255, 0, 0), 2)
        if endX< 380 and endY >= 280:
            inCount+=1
            trackers.pop(num)
            
            
            
            
            
inCount = 0
outCount = 0
Font = cv.FONT_HERSHEY_COMPLEX

vid = cv.VideoCapture('highway.mp4')
while True:
    ret,frame = vid.read()
    frame_o = cv.resize(frame,(640,480))
    
    frame_cropped = frame_o[240:640, 0:640]
    
    
    blob = cv.dnn.blobFromImage(frame_cropped, 1/255.0, (inpWidth, inpHeight), [0, 0, 0], 1, crop=False)
    
    net.setInput(blob)
    outs = net.forward(getOutputNames(net))
    
    postProcess(frame_o, outs)
    cv.putText(frame_o, f"IN : {inCount}",(20,40), Font, 1, (250,0,0), 2 )
    
    cv.imshow("Object Detection", frame_o)
    cv.imshow("Cropped", frame_cropped)
    
    if cv.waitKey(1) & 0xFF == ord('q'):    
        break  

error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\dnn\src\darknet\darknet_importer.cpp:210: error: (-212:Parsing error) Failed to open NetParameter file: yolov4-tiny.cfg in function 'cv::dnn::dnn4_v20240521::readNetFromDarknet'
